# Structured data prediction using Vertex AI Platform


## Learning Objectives

1. Create a BigQuery Dataset and Google Cloud Storage Bucket 
2. Export from BigQuery to CSVs in GCS
3. Training on Cloud AI Platform
4. Deploy trained model

## Introduction

In this notebook, you train, evaluate, and deploy a machine learning model to predict a baby's weight.



In [4]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [5]:
!pip install --user google-cloud-bigquery>=2.26.0

In [6]:
!pip install -U google-cloud-aiplatform "shapely>=2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.103.0
    Uninstalling google-cloud-aiplatform-1.103.0:
      Successfully uninstalled google-cloud-aiplatform-1.103.0


**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

## Set up environment variables and load necessary libraries

Set environment variables so that we can use them throughout the entire notebook. We will be using our project name for our bucket, so you only need to change your project and region.

In [12]:
# change these to try this notebook out
BUCKET = 'gcpmachinelearning' # Replace with the your bucket name
PROJECT = 'qwiklabs-gcp-04-aeacdf2f32ed' # Replace with your project-id
REGION = 'us-central1'

In [13]:
import os

from google.cloud import bigquery

In [14]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.11"
os.environ["PYTHONVERSION"] = "3.7"

In [16]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-04-aeacdf2f32ed


## The source dataset

Our dataset is hosted in [BigQuery](https://cloud.google.com/bigquery/). The CDC's Natality data has details on US births from 1969 to 2008 and is a publically available dataset, meaning anyone with a GCP account has access. Click [here](https://console.cloud.google.com/bigquery?project=bigquery-public-data&p=publicdata&d=samples&t=natality&page=table) to access the dataset.

The natality dataset is relatively large at almost 138 million rows and 31 columns, but simple to understand. `weight_pounds` is the target, the continuous value we’ll train a model to predict.

## Create a BigQuery Dataset and Google Cloud Storage Bucket 

A BigQuery dataset is a container for tables, views, and models built with BigQuery ML. Let's create one called __babyweight__. We'll do the same for a GCS bucket for our project too.

In [17]:
%%bash

# Create a BigQuery dataset for babyweight if it doesn't exist
datasetexists=$(bq ls -d | grep -w babyweight)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset titled: babyweight"
    
    bq --location=US mk --dataset \
        --description "Babyweight" \
        $PROJECT:babyweight
    echo "Here are your current datasets:"
    bq ls
fi
    
## Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
    
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "Here are your current buckets:"
    gsutil ls
fi

Creating BigQuery dataset titled: babyweight
Dataset 'qwiklabs-gcp-04-aeacdf2f32ed:babyweight' successfully created.
Here are your current datasets:
  datasetId   
 ------------ 
  babyweight  
Bucket exists, let's not recreate it.


## Create the training and evaluation data tables

Since there is already a publicly available dataset, we can simply create the training and evaluation data tables using this raw input data. First we are going to create a subset of the data limiting our columns to `weight_pounds`, `is_male`, `mother_age`, `plurality`, and `gestation_weeks` as well as some simple filtering and a column to hash on for repeatable splitting.

* Note:  The dataset in the create table code below is the one created previously, e.g. "babyweight".

### Preprocess and filter dataset

We have some preprocessing and filtering we would like to do to get our data in the right format for training.

Preprocessing:
* Cast `is_male` from `BOOL` to `STRING`
* Cast `plurality` from `INTEGER` to `STRING` where `[1, 2, 3, 4, 5]` becomes `["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]`
* Add `hashcolumn` hashing on `year` and `month`

Filtering:
* Only want data for years later than `2000`
* Only want baby weights greater than `0`
* Only want mothers whose age is greater than `0`
* Only want plurality to be greater than `0`
* Only want the number of weeks of gestation to be greater than `0`

In [18]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data AS
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CASE
        WHEN plurality = 1 THEN "Single(1)"
        WHEN plurality = 2 THEN "Twins(2)"
        WHEN plurality = 3 THEN "Triplets(3)"
        WHEN plurality = 4 THEN "Quadruplets(4)"
        WHEN plurality = 5 THEN "Quintuplets(5)"
    END AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING)
        )
    ) AS hashmonth
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0

Query is running:   0%|          |

""


### Augment dataset to simulate missing data

Now we want to augment our dataset with our simulated babyweight data by setting all gender information to `Unknown` and setting plurality of all non-single births to `Multiple(2+)`.

In [19]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_augmented_data AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data
UNION ALL
SELECT
    weight_pounds,
    "Unknown" AS is_male,
    mother_age,
    CASE
        WHEN plurality = "Single(1)" THEN plurality
        ELSE "Multiple(2+)"
    END AS plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data

Query is running:   0%|          |

""


### Split augmented dataset into train and eval sets

Using `hashmonth`, apply a module to get approximately a 75/25 train-eval split.

#### Split augmented dataset into train dataset

In [20]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_train AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    ABS(MOD(hashmonth, 4)) < 3

Query is running:   0%|          |

""


#### Split augmented dataset into eval dataset

In [21]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_eval AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    ABS(MOD(hashmonth, 4)) = 3

Query is running:   0%|          |

""


## Verify table creation

Verify that you created the dataset and training data table.

In [22]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

Query is running:   0%|          |

Downloading: |          |

,weight_pounds,is_male,mother_age,plurality,gestation_weeks


In [23]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

Query is running:   0%|          |

Downloading: |          |

,weight_pounds,is_male,mother_age,plurality,gestation_weeks


## Export from BigQuery to CSVs in GCS

Use BigQuery Python API to export our train and eval tables to Google Cloud Storage in the CSV format to be used later for TensorFlow/Keras training. We'll want to use the dataset we've been using above as well as repeat the process for both training and evaluation data.

In [25]:
# Construct a BigQuery client object.
client = bigquery.Client()

dataset_name = "babyweight"

# Create dataset reference object
dataset_ref = client.dataset(
    dataset_id=dataset_name, project=client.project)

# Export both train and eval tables
for step in ["train", "eval"]:
    destination_uri = os.path.join(
        "gs://", BUCKET, dataset_name, "data", "{}*.csv".format(step))
    table_name = "babyweight_data_{}".format(step)
    table_ref = dataset_ref.table(table_name)
    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        # Location must match that of the source table.
        location="US",
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print("Exported {}:{}.{} to {}".format(
        client.project, dataset_name, table_name, destination_uri))

Exported qwiklabs-gcp-04-aeacdf2f32ed:babyweight.babyweight_data_train to gs://gcpmachinelearning/babyweight/data/train*.csv
Exported qwiklabs-gcp-04-aeacdf2f32ed:babyweight.babyweight_data_eval to gs://gcpmachinelearning/babyweight/data/eval*.csv


## Verify CSV creation

Verify that we correctly created the CSV files in our bucket.

In [26]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/data/*.csv

gs://gcpmachinelearning/babyweight/data/eval000000000000.csv
gs://gcpmachinelearning/babyweight/data/eval000000000001.csv
gs://gcpmachinelearning/babyweight/data/eval000000000002.csv
gs://gcpmachinelearning/babyweight/data/eval000000000003.csv
gs://gcpmachinelearning/babyweight/data/eval000000000004.csv
gs://gcpmachinelearning/babyweight/data/eval000000000005.csv
gs://gcpmachinelearning/babyweight/data/eval000000000006.csv
gs://gcpmachinelearning/babyweight/data/eval000000000007.csv
gs://gcpmachinelearning/babyweight/data/eval000000000008.csv
gs://gcpmachinelearning/babyweight/data/eval000000000009.csv
gs://gcpmachinelearning/babyweight/data/eval000000000010.csv
gs://gcpmachinelearning/babyweight/data/eval000000000011.csv
gs://gcpmachinelearning/babyweight/data/eval000000000012.csv
gs://gcpmachinelearning/babyweight/data/eval000000000013.csv
gs://gcpmachinelearning/babyweight/data/eval000000000014.csv
gs://gcpmachinelearning/babyweight/data/eval000000000015.csv
gs://gcpmachinelearning/

## Check data exists

Verify that you previously created CSV files we'll be using for training and evaluation.

In [27]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/data/*000000000000.csv

gs://gcpmachinelearning/babyweight/data/eval000000000000.csv
gs://gcpmachinelearning/babyweight/data/train000000000000.csv


## Training on Cloud AI Platform

Now that we see everything is working locally, it's time to train on the cloud! 

To submit to the Cloud we use [`gcloud ai-platform jobs submit training [jobname]`](https://cloud.google.com/sdk/gcloud/reference/ml-engine/jobs/submit/training) and simply specify some additional parameters for AI Platform Training Service:
- jobname: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- job-dir: A GCS location to upload the Python package to
- runtime-version: Version of TF to use.
- python-version: Version of Python to use.
- region: Cloud region to train in. See [here](https://cloud.google.com/ml-engine/docs/tensorflow/regions) for supported AI Platform Training Service regions

Below the `-- \` we add in the arguments for our `task.py` file.

In [29]:
%%bash

OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBID=babyweight_$(date -u +%y%m%d_%H%M%S)

gcloud ai-platform jobs submit training ${JOBID} \
    --region=${REGION} \
    --module-name=trainer.task \
    --package-path=$(pwd)/babyweight/trainer \
    --job-dir=${OUTDIR} \
    --staging-bucket=gs://${BUCKET} \
    --master-machine-type=n1-standard-8 \
    --scale-tier=CUSTOM \
    --runtime-version=${TFVERSION} \
    --python-version=${PYTHONVERSION} \
    -- \
    --train_data_path=gs://${BUCKET}/babyweight/data/train*.csv \
    --eval_data_path=gs://${BUCKET}/babyweight/data/eval*.csv \
    --output_dir=${OUTDIR} \
    --num_epochs=10 \
    --train_examples=10000 \
    --eval_steps=100 \
    --batch_size=32 \
    --nembeds=8

API [ml.googleapis.com] not enabled on project [qwiklabs-gcp-04-aeacdf2f32ed]. 
Would you like to enable and retry (this will take a few minutes)? (y/N)?  
ERROR: (gcloud.ai-platform.jobs.submit.training) [693758598515-compute@developer.gserviceaccount.com] does not have permission to access projects instance [qwiklabs-gcp-04-aeacdf2f32ed] (or it may not exist): AI Platform Training & Prediction API has not been used in project qwiklabs-gcp-04-aeacdf2f32ed before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=qwiklabs-gcp-04-aeacdf2f32ed then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. This command is authenticated as 693758598515-compute@developer.gserviceaccount.com which is the active account specified by the [core/account] property.
AI Platform Training & Prediction API has not been used in project qwiklabs-gcp-04-aeacdf2f32ed before or i

CalledProcessError: Command 'b'\nOUTDIR=gs://${BUCKET}/babyweight/trained_model\nJOBID=babyweight_$(date -u +%y%m%d_%H%M%S)\n\ngcloud ai-platform jobs submit training ${JOBID} \\\n    --region=${REGION} \\\n    --module-name=trainer.task \\\n    --package-path=$(pwd)/babyweight/trainer \\\n    --job-dir=${OUTDIR} \\\n    --staging-bucket=gs://${BUCKET} \\\n    --master-machine-type=n1-standard-8 \\\n    --scale-tier=CUSTOM \\\n    --runtime-version=${TFVERSION} \\\n    --python-version=${PYTHONVERSION} \\\n    -- \\\n    --train_data_path=gs://${BUCKET}/babyweight/data/train*.csv \\\n    --eval_data_path=gs://${BUCKET}/babyweight/data/eval*.csv \\\n    --output_dir=${OUTDIR} \\\n    --num_epochs=10 \\\n    --train_examples=10000 \\\n    --eval_steps=100 \\\n    --batch_size=32 \\\n    --nembeds=8\n'' returned non-zero exit status 1.

The training job should complete within 15 to 20 minutes. You do not need to wait for this training job to finish before moving forward in the notebook, but will need a trained model.

## Check our trained model files

Let's check the directory structure of our outputs of our trained model in folder we exported. We'll want to deploy the saved_model.pb within the timestamped directory as well as the variable values in the variables folder. Therefore, we need the path of the timestamped directory so that everything within it can be found by Cloud AI Platform's model deployment service.

In [18]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model

gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/
gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/20210925073734/
gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/checkpoints/


In [19]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \
                 | tail -1)
gsutil ls ${MODEL_LOCATION}

gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/20210925073734/
gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/20210925073734/saved_model.pb
gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/20210925073734/assets/
gs://qwiklabs-gcp-02-2372fbdc4b9d/babyweight/trained_model/20210925073734/variables/


## Deploy trained model

Deploying the trained model to act as a REST web service is a simple gcloud call.

In [20]:
%%bash
gcloud config set ai_platform/region global


Updated property [ai_platform/region].


In [30]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \
                 | tail -1 | tr -d '[:space:]')
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"
# gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
# gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.6 \
    --python-version=3.7

CommandException: One or more URLs matched no objects.


Deleting and deploying babyweight ml_on_gcp from 


Using endpoint [https://ml.googleapis.com/]
API [ml.googleapis.com] not enabled on project [qwiklabs-gcp-04-aeacdf2f32ed]. 
Would you like to enable and retry (this will take a few minutes)? (y/N)?  Please enter 'y' or 'n':  Please enter 'y' or 'n':  Please enter 'y' or 'n':  Please enter 'y' or 'n':  Please enter 'y' or 'n':  
ERROR: (gcloud.ai-platform.models.create) [693758598515-compute@developer.gserviceaccount.com] does not have permission to access projects instance [qwiklabs-gcp-04-aeacdf2f32ed] (or it may not exist): AI Platform Training & Prediction API has not been used in project qwiklabs-gcp-04-aeacdf2f32ed before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=qwiklabs-gcp-04-aeacdf2f32ed then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. This command is authenticated as 693758598515-compute@developer.gserviceaccount.com which is t

CalledProcessError: Command 'b'MODEL_NAME="babyweight"\nMODEL_VERSION="ml_on_gcp"\nMODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \\\n                 | tail -1 | tr -d \'[:space:]\')\necho "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"\n# gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}\n# gcloud ai-platform models delete ${MODEL_NAME}\ngcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}\ngcloud ai-platform versions create ${MODEL_VERSION} \\\n    --model=${MODEL_NAME} \\\n    --origin=${MODEL_LOCATION} \\\n    --runtime-version=2.6 \\\n    --python-version=3.7\n'' returned non-zero exit status 1.

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.